In [5]:
from __future__ import print_function
import IPython
import sys
from music21 import *
import numpy as np
from grammar import *
from qa import *
from preprocess import * 
from music_utils import *
from data_utils import *
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

# each value in X are notes that comprises a pitch and a duration 
# shape of X: (60,30,78)
# Number of training examples 60
# length of sequence 30
# number of unique value 78
X, Y, n_values, indices_values = load_music_utils()

# dimension of the hidden state = 64
n_a = 64 

# layer objects (global variables)
reshapor = Reshape((1, 78))                        
LSTM_cell = LSTM(n_a, return_state = True) # takes positive integer which represents the dimentionality of the output space       
densor = Dense(n_values, activation='softmax') 

# 
def djmodel(Tx, n_a, n_values):
    """
    Implement the model
    
    Arguments:
    Tx -- length of the sequence in a corpus
    n_a -- the number of activations used in our model
    n_values -- number of unique values in the music data 
    
    Returns:
    model -- a keras model instance
    """
    
    # input of the model with a shape of 30x78 
    X = Input(shape=(Tx, n_values))
    
    # initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    
    outputs = [] 
    
    for t in range(Tx):
        x = Lambda(lambda x: X[:,t,:])(X)
        x = reshapor(x)
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        out = densor(a)
        outputs.append(out)

    model = Model(inputs=[X, a0, c0], outputs=outputs)
    
    return model

model = djmodel(Tx = 30 , n_a = 64, n_values = 78)

opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

# configure the model for training
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

m = 60
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

# Trains the model for a given number of epochs
model.fit([X, a0, c0], list(Y), epochs=100)

def music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 100):
    """
    Uses the trained "LSTM_cell" and "densor" from model() to generate a sequence of values.
    
    Arguments:
    LSTM_cell -- the trained "LSTM_cell" from model(), Keras layer object
    densor -- the trained "densor" from model(), Keras layer object
    n_values -- integer, number of unique values
    n_a -- number of units in the LSTM_cell
    Ty -- integer, number of time steps to generate
    
    Returns:
    inference_model -- Keras model instance
    """
    
    # Define the input of our model with a shape 
    x0 = Input(shape=(1, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0

    outputs = []
    
    # Loop over Ty and generate a value at every time step
    for t in range(Ty):
        
        # Perform one step of LSTM_cell
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        
        # Apply Dense layer to the hidden state output of the LSTM_cell
        out = densor(a)

        # Append the prediction "out" to "outputs". out.shape = (None, 78) 
        outputs.append(out)
        x = Lambda(one_hot)(out)
        

    inference_model = Model(inputs=[x0, a0, c0], outputs=outputs)
    
    return inference_model

inference_model = music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 50)



out_stream = generate_music(inference_model)

Epoch 1/100
60/60 [==============================] - 14s 230ms/step - loss: 125.6417 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0167 - dense_2_acc_1: 0.0333 - dense_2_acc_2: 0.0667 - dense_2_acc_3: 0.0833 - dense_2_acc_4: 0.0667 - dense_2_acc_5: 0.1000 - dense_2_acc_6: 0.0333 - dense_2_acc_7: 0.0833 - dense_2_acc_8: 0.0667 - dense_2_acc_9: 0.0833 - dense_2_acc_10: 0.0833 - dense_2_acc_11: 0.0500 - dense_2_acc_12: 0.1167 - dense_2_acc_13: 0.2167 - dense_2_acc_14: 0.0500 - dense_2_acc_15: 0.0833 - dense_2_acc_16: 0.0833 - dense_2_acc_17: 0.1000 - dense_2_acc_18: 0.1500 - dense_2_acc_19: 0.1000 - dense_2_acc_20: 0.1167 - dense_2_acc_21: 0.1167 - dense_2_acc_22: 0.1667 - dense_2_acc_23: 0.1167 - dense_2_acc_24: 0.0333 - dense_2_acc_25: 0.1167 - dense_2_acc_26: 0.0500 - dense_2_acc_27: 0.0333 - dense_2_acc_28: 0.1500 - dense_2_acc_29: 0.0500                                                              
Epoch 2/100
60/60 [==============================] - 0s 806us/step - loss: 121.8418 - de

60/60 [==============================] - 0s 838us/step - loss: 92.6648 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0667 - dense_2_acc_1: 0.2167 - dense_2_acc_2: 0.2167 - dense_2_acc_3: 0.2167 - dense_2_acc_4: 0.2833 - dense_2_acc_5: 0.1333 - dense_2_acc_6: 0.1333 - dense_2_acc_7: 0.2333 - dense_2_acc_8: 0.2000 - dense_2_acc_9: 0.2167 - dense_2_acc_10: 0.1833 - dense_2_acc_11: 0.1167 - dense_2_acc_12: 0.1667 - dense_2_acc_13: 0.1833 - dense_2_acc_14: 0.2667 - dense_2_acc_15: 0.1500 - dense_2_acc_16: 0.1833 - dense_2_acc_17: 0.0833 - dense_2_acc_18: 0.2167 - dense_2_acc_19: 0.0667 - dense_2_acc_20: 0.1500 - dense_2_acc_21: 0.1667 - dense_2_acc_22: 0.1667 - dense_2_acc_23: 0.2167 - dense_2_acc_24: 0.1000 - dense_2_acc_25: 0.2333 - dense_2_acc_26: 0.1333 - dense_2_acc_27: 0.1667 - dense_2_acc_28: 0.1833 - dense_2_acc_29: 0.0000e+00
Epoch 11/100
60/60 [==============================] - 0s 808us/step - loss: 88.5143 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0667 - dense_2_acc_1: 0.2167 - d

60/60 [==============================] - 0s 747us/step - loss: 62.4189 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0667 - dense_2_acc_1: 0.3000 - dense_2_acc_2: 0.3667 - dense_2_acc_3: 0.3000 - dense_2_acc_4: 0.4167 - dense_2_acc_5: 0.4000 - dense_2_acc_6: 0.3833 - dense_2_acc_7: 0.5000 - dense_2_acc_8: 0.4000 - dense_2_acc_9: 0.4500 - dense_2_acc_10: 0.4833 - dense_2_acc_11: 0.4167 - dense_2_acc_12: 0.5333 - dense_2_acc_13: 0.5500 - dense_2_acc_14: 0.4333 - dense_2_acc_15: 0.3167 - dense_2_acc_16: 0.4667 - dense_2_acc_17: 0.3500 - dense_2_acc_18: 0.5500 - dense_2_acc_19: 0.4667 - dense_2_acc_20: 0.4667 - dense_2_acc_21: 0.4833 - dense_2_acc_22: 0.5667 - dense_2_acc_23: 0.4667 - dense_2_acc_24: 0.4000 - dense_2_acc_25: 0.5000 - dense_2_acc_26: 0.5667 - dense_2_acc_27: 0.5000 - dense_2_acc_28: 0.5000 - dense_2_acc_29: 0.0167
Epoch 20/100
60/60 [==============================] - 0s 867us/step - loss: 59.2046 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0667 - dense_2_acc_1: 0.3000 - dense

60/60 [==============================] - 0s 766us/step - loss: 38.3862 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0333 - dense_2_acc_1: 0.4000 - dense_2_acc_2: 0.5500 - dense_2_acc_3: 0.4500 - dense_2_acc_4: 0.6833 - dense_2_acc_5: 0.7500 - dense_2_acc_6: 0.8167 - dense_2_acc_7: 0.7833 - dense_2_acc_8: 0.8000 - dense_2_acc_9: 0.8333 - dense_2_acc_10: 0.8500 - dense_2_acc_11: 0.8667 - dense_2_acc_12: 0.9167 - dense_2_acc_13: 0.9167 - dense_2_acc_14: 0.8333 - dense_2_acc_15: 0.8167 - dense_2_acc_16: 0.9000 - dense_2_acc_17: 0.8500 - dense_2_acc_18: 0.8500 - dense_2_acc_19: 0.8500 - dense_2_acc_20: 0.8667 - dense_2_acc_21: 0.9167 - dense_2_acc_22: 0.8167 - dense_2_acc_23: 0.8500 - dense_2_acc_24: 0.8000 - dense_2_acc_25: 0.9167 - dense_2_acc_26: 0.8000 - dense_2_acc_27: 0.8833 - dense_2_acc_28: 0.8333 - dense_2_acc_29: 0.0167
Epoch 29/100
60/60 [==============================] - 0s 770us/step - loss: 36.0925 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0333 - dense_2_acc_1: 0.4000 - dense

60/60 [==============================] - 0s 715us/step - loss: 21.9921 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0333 - dense_2_acc_1: 0.4167 - dense_2_acc_2: 0.6667 - dense_2_acc_3: 0.6667 - dense_2_acc_4: 0.8167 - dense_2_acc_5: 0.9333 - dense_2_acc_6: 0.9333 - dense_2_acc_7: 0.9500 - dense_2_acc_8: 0.9500 - dense_2_acc_9: 1.0000 - dense_2_acc_10: 0.9833 - dense_2_acc_11: 1.0000 - dense_2_acc_12: 1.0000 - dense_2_acc_13: 1.0000 - dense_2_acc_14: 1.0000 - dense_2_acc_15: 1.0000 - dense_2_acc_16: 1.0000 - dense_2_acc_17: 1.0000 - dense_2_acc_18: 1.0000 - dense_2_acc_19: 1.0000 - dense_2_acc_20: 1.0000 - dense_2_acc_21: 1.0000 - dense_2_acc_22: 1.0000 - dense_2_acc_23: 1.0000 - dense_2_acc_24: 1.0000 - dense_2_acc_25: 1.0000 - dense_2_acc_26: 0.9833 - dense_2_acc_27: 0.9833 - dense_2_acc_28: 0.9833 - dense_2_acc_29: 0.0333
Epoch 38/100
60/60 [==============================] - 0s 728us/step - loss: 20.7182 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0333 - dense_2_acc_1: 0.4333 - dense

...

60/60 [==============================] - 0s 581us/step - loss: 8.1724 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0333 - dense_2_acc_1: 0.5833 - dense_2_acc_2: 0.8833 - dense_2_acc_3: 1.0000 - dense_2_acc_4: 0.9833 - dense_2_acc_5: 1.0000 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 1.0000 - dense_2_acc_8: 1.0000 - dense_2_acc_9: 1.0000 - dense_2_acc_10: 1.0000 - dense_2_acc_11: 1.0000 - dense_2_acc_12: 1.0000 - dense_2_acc_13: 1.0000 - dense_2_acc_14: 1.0000 - dense_2_acc_15: 1.0000 - dense_2_acc_16: 1.0000 - dense_2_acc_17: 1.0000 - dense_2_acc_18: 1.0000 - dense_2_acc_19: 1.0000 - dense_2_acc_20: 1.0000 - dense_2_acc_21: 1.0000 - dense_2_acc_22: 1.0000 - dense_2_acc_23: 1.0000 - dense_2_acc_24: 1.0000 - dense_2_acc_25: 1.0000 - dense_2_acc_26: 1.0000 - dense_2_acc_27: 1.0000 - dense_2_acc_28: 1.0000 - dense_2_acc_29: 0.0333
Epoch 65/100
60/60 [==============================] - 0s 688us/step - loss: 8.0427 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0333 - dense_2_acc_1: 0.6000 - dense_2

60/60 [==============================] - 0s 788us/step - loss: 7.2310 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.1000 - dense_2_acc_1: 0.6500 - dense_2_acc_2: 0.9000 - dense_2_acc_3: 1.0000 - dense_2_acc_4: 0.9833 - dense_2_acc_5: 1.0000 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 1.0000 - dense_2_acc_8: 1.0000 - dense_2_acc_9: 1.0000 - dense_2_acc_10: 1.0000 - dense_2_acc_11: 1.0000 - dense_2_acc_12: 1.0000 - dense_2_acc_13: 1.0000 - dense_2_acc_14: 1.0000 - dense_2_acc_15: 1.0000 - dense_2_acc_16: 1.0000 - dense_2_acc_17: 1.0000 - dense_2_acc_18: 1.0000 - dense_2_acc_19: 1.0000 - dense_2_acc_20: 1.0000 - dense_2_acc_21: 1.0000 - dense_2_acc_22: 1.0000 - dense_2_acc_23: 1.0000 - dense_2_acc_24: 1.0000 - dense_2_acc_25: 1.0000 - dense_2_acc_26: 1.0000 - dense_2_acc_27: 1.0000 - dense_2_acc_28: 1.0000 - dense_2_acc_29: 0.0333
Epoch 74/100
60/60 [==============================] - 0s 710us/step - loss: 7.1493 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.1000 - dense_2_acc_1: 0.6500 - dense_2

60/60 [==============================] - 0s 733us/step - loss: 6.6336 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.1000 - dense_2_acc_1: 0.6500 - dense_2_acc_2: 0.9167 - dense_2_acc_3: 1.0000 - dense_2_acc_4: 1.0000 - dense_2_acc_5: 1.0000 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 1.0000 - dense_2_acc_8: 1.0000 - dense_2_acc_9: 1.0000 - dense_2_acc_10: 1.0000 - dense_2_acc_11: 1.0000 - dense_2_acc_12: 1.0000 - dense_2_acc_13: 1.0000 - dense_2_acc_14: 1.0000 - dense_2_acc_15: 1.0000 - dense_2_acc_16: 1.0000 - dense_2_acc_17: 1.0000 - dense_2_acc_18: 1.0000 - dense_2_acc_19: 1.0000 - dense_2_acc_20: 1.0000 - dense_2_acc_21: 1.0000 - dense_2_acc_22: 1.0000 - dense_2_acc_23: 1.0000 - dense_2_acc_24: 1.0000 - dense_2_acc_25: 1.0000 - dense_2_acc_26: 1.0000 - dense_2_acc_27: 1.0000 - dense_2_acc_28: 1.0000 - dense_2_acc_29: 0.0333
Epoch 83/100
60/60 [==============================] - 0s 786us/step - loss: 6.5785 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.1000 - dense_2_acc_1: 0.6500 - dense_2

60/60 [==============================] - 0s 709us/step - loss: 6.2171 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.1000 - dense_2_acc_1: 0.6500 - dense_2_acc_2: 0.9333 - dense_2_acc_3: 1.0000 - dense_2_acc_4: 1.0000 - dense_2_acc_5: 1.0000 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 1.0000 - dense_2_acc_8: 1.0000 - dense_2_acc_9: 1.0000 - dense_2_acc_10: 1.0000 - dense_2_acc_11: 1.0000 - dense_2_acc_12: 1.0000 - dense_2_acc_13: 1.0000 - dense_2_acc_14: 1.0000 - dense_2_acc_15: 1.0000 - dense_2_acc_16: 1.0000 - dense_2_acc_17: 1.0000 - dense_2_acc_18: 1.0000 - dense_2_acc_19: 1.0000 - dense_2_acc_20: 1.0000 - dense_2_acc_21: 1.0000 - dense_2_acc_22: 1.0000 - dense_2_acc_23: 1.0000 - dense_2_acc_24: 1.0000 - dense_2_acc_25: 1.0000 - dense_2_acc_26: 1.0000 - dense_2_acc_27: 1.0000 - dense_2_acc_28: 1.0000 - dense_2_acc_29: 0.0333
Epoch 92/100
60/60 [==============================] - 0s 708us/step - loss: 6.1772 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.1000 - dense_2_acc_1: 0.6500 - dense_2

60/60 [==============================] - 0s 756us/step - loss: 5.9088 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.1000 - dense_2_acc_1: 0.6667 - dense_2_acc_2: 0.9500 - dense_2_acc_3: 1.0000 - dense_2_acc_4: 1.0000 - dense_2_acc_5: 1.0000 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 1.0000 - dense_2_acc_8: 1.0000 - dense_2_acc_9: 1.0000 - dense_2_acc_10: 1.0000 - dense_2_acc_11: 1.0000 - dense_2_acc_12: 1.0000 - dense_2_acc_13: 1.0000 - dense_2_acc_14: 1.0000 - dense_2_acc_15: 1.0000 - dense_2_acc_16: 1.0000 - dense_2_acc_17: 1.0000 - dense_2_acc_18: 1.0000 - dense_2_acc_19: 1.0000 - dense_2_acc_20: 1.0000 - dense_2_acc_21: 1.0000 - dense_2_acc_22: 1.0000 - dense_2_acc_23: 1.0000 - dense_2_acc_24: 1.0000 - dense_2_acc_25: 1.0000 - dense_2_acc_26: 1.0000 - dense_2_acc_27: 1.0000 - dense_2_acc_28: 1.0000 - dense_2_acc_29: 0.0333
Predicting new values for different set of chords.
Generated 51 sounds using the predicted values for the set of chords ("1") and after pruning
Generated 50 sounds u